#Question 1

Algorithm 1



In [8]:
import networkx as nx
from typing import Hashable

# Mongolian tent graph builder

def build_mongolian_tent_3n(n: int) -> nx.Graph:
    G = nx.Graph()
    # Create grid nodes (i,j)
    for i in range(3):
        for j in range(n):
            G.add_node((i, j))

    # Add edges for the grid
    for i in range(3):
        for j in range(n):
            if j + 1 < n:
                G.add_edge((i, j), (i, j + 1))  # horizontal
            if i + 1 < 3:
                G.add_edge((i, j), (i + 1, j))  # vertical

    # Add apex node
    apex = "Apex"
    G.add_node(apex)

    # Connect apex to all top row nodes (i = 0, j = 0..n-1)
    for j in range(n):
        G.add_edge(apex, (0, j))

    return G


# verify labeling (edge sums all distinct)


def verify_labeling(G: nx.Graph, labels: dict[Hashable, int]) -> bool:
    """
    Check that:
      - all vertices are labeled,
      - all edge sums label[u] + label[v] are distinct.
    """
    if len(labels) != G.number_of_nodes():
        return False

    seen_sums = set()
    for u, v in G.edges():
        if u not in labels or v not in labels:
            return False
        s = labels[u] + labels[v]
        if s in seen_sums:
            return False
        seen_sums.add(s)
    return True


# Main algorithm: weights in ascending order, k emerges

def label_graph_with_ascending_weights(
    G: nx.Graph,
    max_sum: int | None = None,
    seed_vertex: Hashable | None = None,
    seed_label: int = 1,
) -> dict[Hashable, int] | None:
    """
    Try to label G such that:
        - each vertex gets a positive integer label,
        - each edge (u, v) has weight = label[u] + label[v],
        - all edge weights are DISTINCT.

    Strategy:
        - We do NOT choose k beforehand.
        - We keep a global weight w starting from 2 and going up.
        - For each w:
            * try to assign it to some frontier edge (u, v)
              with exactly one labeled endpoint, deriving the
              new vertex label from w.
            * if no edge can use w, we skip w and go to w+1.
        - Every time we label a new vertex v, we automatically
          create sums for all edges (v, neighbor) where neighbor
          is already labeled, and we ensure those sums are also
          unique.
        - Once all vertices are labeled, k = max(labels.values())
          is whatever it turns out to be.
    """

    n = G.number_of_nodes()
    m = G.number_of_edges()

    if n == 0:
        return {}

    # Choose seed vertex if not provided: pick a high-degree vertex
    if seed_vertex is None:
        seed_vertex = max(G.degree(), key=lambda x: x[1])[0]

    # If max_sum not given, set a big-ish upper bound
    if max_sum is None:
        max_sum = max(4 * m, 10)

    # State
    labels: dict[Hashable, int] = {}
    used_vertex_labels: set[int] = set()
    used_sums: set[int] = set()

    # Initialize seed
    labels[seed_vertex] = seed_label
    used_vertex_labels.add(seed_label)

    # Backtracking core

    def backtrack(next_weight: int) -> bool:
        # If all vertices labeled, we are done
        if len(labels) == n:
            return verify_labeling(G, labels)

        # If we ran out of allowed weights, fail
        if next_weight > max_sum:
            return False

        # Ensure next_weight is not already used as a sum
        w = next_weight
        while w in used_sums:
            w += 1
            if w > max_sum:
                return False

        # Collect frontier edges: (u, v) where exactly one endpoint is labeled
        frontier_edges: list[tuple[Hashable, Hashable]] = []
        for u in labels:
            for v in G[u]:
                if v not in labels:
                    frontier_edges.append((u, v))

        # If there are unlabeled vertices but no frontier edges, graph is disconnected
        if not frontier_edges:
            return False

        # Heuristic: focus on vertices with higher degree
        frontier_edges.sort(key=lambda e: -G.degree()[e[1]])

        # Try to use weight w on some frontier edge
        used_somewhere = False

        for (u, v) in frontier_edges:
            lu = labels[u]
            x = w - lu  # candidate label for v

            # Labels must be positive integers and not reused
            if x <= 0:
                continue
            if x in used_vertex_labels:
                continue

            # Check consistency with all already-labeled neighbors of v
            new_sums: list[int] = []
            ok = True
            for nbr in G[v]:
                if nbr in labels:
                    s = x + labels[nbr]
                    if s in used_sums or s in new_sums:
                        ok = False
                        break
                    new_sums.append(s)

            if not ok:
                continue

            # We can use weight w on edge (u, v) by giving v the label x
            used_somewhere = True

            # Choose
            labels[v] = x
            used_vertex_labels.add(x)
            for s in new_sums:
                used_sums.add(s)

            # Recurse with next weight (w+1)
            if backtrack(w + 1):
                return True

            # Undo
            del labels[v]
            used_vertex_labels.remove(x)
            for s in new_sums:
                used_sums.remove(s)

        # If we could not use weight w on any frontier edge,
        # we simply skip w and move to w+1 with the same labels.
        if not used_somewhere:
            return backtrack(w + 1)

        return False


    success = backtrack(2)
    if success:
        return labels
    else:
        return None

# Example usage

if __name__ == "__main__":
    n = 10
    G = build_mongolian_tent_3n(n)

    labels = label_graph_with_ascending_weights(G, max_sum=500)

    if labels is None:
        print("No labeling found within the given weight range.")
    else:
        print("Vertex labels:")

        # Safe sort: put "Apex" first, then grid nodes sorted by (row, col)
        def sort_key(v: Hashable):
            if v == "Apex":
                return (0, -1, -1)   # first
            if isinstance(v, tuple):
                i, j = v
                return (1, i, j)    # then by row, then column
            return (2, 0, 0)        # anything else at the end

        for v in sorted(labels.keys(), key=sort_key):
            print(f"  {v} -> {labels[v]}")

        # Show all edge sums (all should be distinct)
        edge_sums = {}
        for u, v in G.edges():
            s = labels[u] + labels[v]
            edge_sums[(u, v)] = s
            ok = verify_labeling(G, labels)
        print("Unique weights:", ok)
        print("k =", max(labels.values()))


Vertex labels:
  Apex -> 1
  (0, 0) -> 20
  (0, 1) -> 2
  (0, 2) -> 3
  (0, 3) -> 5
  (0, 4) -> 6
  (0, 5) -> 8
  (0, 6) -> 9
  (0, 7) -> 11
  (0, 8) -> 12
  (0, 9) -> 25
  (1, 0) -> 26
  (1, 1) -> 13
  (1, 2) -> 15
  (1, 3) -> 14
  (1, 4) -> 10
  (1, 5) -> 17
  (1, 6) -> 21
  (1, 7) -> 23
  (1, 8) -> 19
  (1, 9) -> 16
  (2, 0) -> 30
  (2, 1) -> 27
  (2, 2) -> 18
  (2, 3) -> 29
  (2, 4) -> 22
  (2, 5) -> 31
  (2, 6) -> 28
  (2, 7) -> 32
  (2, 8) -> 33
  (2, 9) -> 34
Unique weights: True
k = 34


Algorithm 2

In [4]:
import networkx as nx
from typing import Hashable, Dict, Set

# Mongolian tent MT(3, n)

def build_mongolian_tent_3n(n: int) -> nx.Graph:
    G = nx.Graph()
    # grid vertices
    for i in range(3):
        for j in range(n):
            G.add_node((i, j))
    # grid edges (horizontal + vertical)
    for i in range(3):
        for j in range(n):
            if j + 1 < n:
                G.add_edge((i, j), (i, j + 1))  # horizontal
            if i + 1 < 3:
                G.add_edge((i, j), (i + 1, j))  # vertical
    # apex
    G.add_node("Apex")
    for j in range(n):
        G.add_edge("Apex", (0, j))
    return G


# Verify labeling: all vertices labeled + edge sums unique

def verify_labeling(G: nx.Graph, labels: Dict[Hashable, int]) -> bool:
    if len(labels) != G.number_of_nodes():
        return False
    seen: Set[int] = set()
    for u, v in G.edges():
        if u not in labels or v not in labels:
            return False
        s = labels[u] + labels[v]
        if s in seen:
            return False
        seen.add(s)
    return True


# Base pattern from your n = 5 solution

BASE_N = 5

BASE_TOP = [14, 2, 3, 5, 12]
BASE_MID = [14, 8, 11, 13, 14]
BASE_BOT = [6, 1, 1, 10, 11]


def compute_base_grid_sums() -> int:
    """
    Compute all edge sums for the 3x5 grid using the base pattern,
    ignoring the apex. Return max sum among grid edges.

    Also assert that within this single block, all grid sums are unique.
    """
    G5 = build_mongolian_tent_3n(BASE_N)

    # Build labels for n=5 using the base pattern (grid only)
    labels_block: Dict[Hashable, int] = {}
    for j in range(BASE_N):
        labels_block[(0, j)] = BASE_TOP[j]
        labels_block[(1, j)] = BASE_MID[j]
        labels_block[(2, j)] = BASE_BOT[j]

    seen: Set[int] = set()
    max_sum = 0
    # only grid edges: skip any edge touching "Apex"
    for u, v in G5.edges():
        if "Apex" in (u, v):
            continue
        s = labels_block[u] + labels_block[v]
        if s in seen:
            raise ValueError(
                f"Base pattern has duplicate grid edge sum {s} on edge {u}-{v}"
            )
        seen.add(s)
        max_sum = max(max_sum, s)

    return max_sum


# Precompute base max sum and choose offset C
BASE_MAX_SUM = compute_base_grid_sums()
OFFSET_C = BASE_MAX_SUM + 1
# Each 3x5 block b will have its sums shifted by b * OFFSET_C, so
# grid-edge sums from different blocks cannot collide.

# 4) Helper: find minimal valid apex label

def smallest_valid_apex(
    top_labels: list[int],
    used_sums: Set[int],
    max_try: int = 10000
) -> int:
    """
    Find the smallest A in [1, max_try] such that
        - for all j, A + top_labels[j] are distinct from used_sums
        - and distinct among themselves (automatic if top_labels are distinct).
    """
    for A in range(1, max_try + 1):
        seen_local: Set[int] = set()
        ok = True
        for t in top_labels:
            s = A + t
            if s in used_sums or s in seen_local:
                ok = False
                break
            seen_local.add(s)
        if ok:
            return A
    raise ValueError("No valid apex label found within the search range.")


# Construct labels for MT(3, n) by repeating the base block with offsets, then choosing minimal apex

def construct_pattern_labels(n: int) -> Dict[Hashable, int]:
    """
    Construct a labeling for MT(3, n) by:
      - Repeating the 3x5 base pattern across the n columns.
      - For block b (columns 5b .. 5b+4) we add offset b*C to all 3 labels,
        where C = BASE_MAX_SUM + 1.
        This makes each block's grid-edge sums live in disjoint intervals.
      - Then choose the *smallest possible* apex label A such that
        all A + top[j] sums are fresh (not in grid_sums) and distinct.

    Vertex labels may repeat.
    All edge sums (including apex edges) are unique.
    """

    labels: Dict[Hashable, int] = {}

    num_blocks = (n + BASE_N - 1) // BASE_N  # ceil(n/5)

    # label the 3×n grid using base pattern + offsets
    for j in range(n):
        b = j // BASE_N            # block index
        idx = j % BASE_N           # position inside block 0..4
        offset = b * OFFSET_C

        labels[(0, j)] = BASE_TOP[idx] + offset
        labels[(1, j)] = BASE_MID[idx] + offset
        labels[(2, j)] = BASE_BOT[idx] + offset

    # Compute all grid-edge sums to know which sums are already taken
    G = build_mongolian_tent_3n(n)
    grid_sums: Set[int] = set()
    for u, v in G.edges():
        if "Apex" in (u, v):
            continue  # skip apex edges for now
        s = labels[u] + labels[v]
        grid_sums.add(s)

    # Choose minimal valid apex label
    top_labels = [labels[(0, j)] for j in range(n)]
    apex_label = smallest_valid_apex(top_labels, grid_sums, max_try=10000)
    labels["Apex"] = apex_label

    return labels

if __name__ == "__main__":
    n=10
    print(f"MT(3, {n})")
    G = build_mongolian_tent_3n(n)
    labels = construct_pattern_labels(n)

    def sort_key(v: Hashable):
        if v == "Apex":
            return (0, -1, -1)
        if isinstance(v, tuple):
            i, j = v
            return (1 + i, j, 0)
        return (4, 0, 0)

    for v in sorted(labels.keys(), key=sort_key):
        print(f"  {v} -> {labels[v]}")

    ok = verify_labeling(G, labels)
    print("Unique weights:", ok)
    print("k =", max(labels.values()))


MT(3, 10)
  Apex -> 1
  (0, 0) -> 14
  (0, 1) -> 2
  (0, 2) -> 3
  (0, 3) -> 5
  (0, 4) -> 12
  (0, 5) -> 43
  (0, 6) -> 31
  (0, 7) -> 32
  (0, 8) -> 34
  (0, 9) -> 41
  (1, 0) -> 14
  (1, 1) -> 8
  (1, 2) -> 11
  (1, 3) -> 13
  (1, 4) -> 14
  (1, 5) -> 43
  (1, 6) -> 37
  (1, 7) -> 40
  (1, 8) -> 42
  (1, 9) -> 43
  (2, 0) -> 6
  (2, 1) -> 1
  (2, 2) -> 1
  (2, 3) -> 10
  (2, 4) -> 11
  (2, 5) -> 35
  (2, 6) -> 30
  (2, 7) -> 30
  (2, 8) -> 39
  (2, 9) -> 40
Unique weights: True
k = 43
